In [1]:
import nltk
from nltk.corpus import sentiwordnet as swn
nltk.download('sentiwordnet')
from pycorenlp import StanfordCoreNLP
from nltk.corpus import wordnet as wn
nltk.download('wordnet_ic')
import requests



[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/arifirmanto/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/arifirmanto/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [13]:
breakdown = swn.senti_synset('exceptional.s.03')

In [14]:
print(breakdown)

<exceptional.s.03: PosScore=0.0 NegScore=0.375>


In [25]:
sentence = 'The service is excellent but the price is expensive'
def babelfy(sentence):
    response = []
    token_word = nltk.word_tokenize(sentence)
    url = 'https://babelfy.io/v1/disambiguate?text='+sentence+'&annRes=WN&lang=en&key=76787a80-1771-41d6-8879-2e5064008923'
    r = requests.get(url)
    
    res = r.json()
    
    for x in res:
        tokenFragment = x['tokenFragment']
        startTknFragment = tokenFragment['start']
        endTknFragment = tokenFragment['end']
        babelSynsetID = x['babelSynsetID'];
        response.append((token_word[startTknFragment], babel_info(babelSynsetID)))
    
    return response

In [26]:
def babel_info(synset_id):
    url = 'https://babelnet.io/v5/getSynset?id='+synset_id+'&key=76787a80-1771-41d6-8879-2e5064008923'
    r = requests.get(url)
    res = r.json()
    return '.'.join(res['mainSense'].split('#'))
    

In [38]:
def get_score(sentence):
    resp = babelfy(sentence)
    ss = []
    for (w, sy) in resp:
        swn_senti = swn.senti_synset(sy)
        ss.append((w, swn_senti.pos_score(), swn_senti.neg_score() ))
        
    return ss

In [39]:
all_scores = get_score(sentence)

In [40]:
all_scores

[('service', 0.0, 0.0),
 ('excellent', 1.0, 0.0),
 ('price', 0.0, 0.0),
 ('expensive', 0.5, 0.0)]